In [25]:
import pandas as pd
import tensorflow as tf
import torch
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset

In [26]:
df = pd.read_csv("Resources/processed_data.csv")
df.head()

,Unnamed: 0,sentiment,text
0,0,Irrelevant,mentioned facebook struggling motivation go ru...
1,1,Neutral,bbc news amazon bos jeff bezos reject claim co...
2,2,Negative,microsoft pay word function poorly samsungus c...
3,3,Negative,csgo matchmaking full closet hacking truly awf...
4,4,Neutral,president slapping american face really commit...


In [27]:
df = df.drop(['Unnamed: 0'],axis=1)

In [28]:
#df_train = pd.get_dummies(df_train, columns=['sentiment'], prefix='sentiment')

In [29]:
df.dropna(subset=['text'],inplace=True)
df.isna().sum()

sentiment    0
text         0
dtype: int64

In [30]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:
def process_text(text):
    try:
        inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", return_attention_mask=True)
        inputs = {key: value.to(model.device) for key, value in inputs.items()}
        return inputs
    except Exception as e:
        # Log the exception and handle it gracefully
        print(f"Error processing text: {e}")
        return None


In [32]:
df["input_data"] = df["text"].apply(process_text)

In [50]:
df.reset_index

<bound method DataFrame.reset_index of         sentiment  ...                                         input_data
0      Irrelevant  ...  {'input_ids': [[tensor(101), tensor(3855), ten...
1         Neutral  ...  {'input_ids': [[tensor(101), tensor(4035), ten...
2        Negative  ...  {'input_ids': [[tensor(101), tensor(7513), ten...
3        Negative  ...  {'input_ids': [[tensor(101), tensor(20116), te...
4         Neutral  ...  {'input_ids': [[tensor(101), tensor(2343), ten...
...           ...  ...                                                ...
74991    Positive  ...  {'input_ids': [[tensor(101), tensor(3651), ten...
74992    Positive  ...  {'input_ids': [[tensor(101), tensor(3651), ten...
74993    Positive  ...  {'input_ids': [[tensor(101), tensor(3651), ten...
74994    Positive  ...  {'input_ids': [[tensor(101), tensor(3651), ten...
74995    Positive  ...  {'input_ids': [[tensor(101), tensor(2066), ten...

[73437 rows x 3 columns]>

In [51]:
X_train, X_test, y_train, y_test = train_test_split(df["input_data"], df["sentiment"], test_size=0.2, random_state=42)


model = AutoModel.from_pretrained(model_name, num_labels=4)
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=32,
    num_train_epochs=3,
    evaluation_strategy="steps",
    save_steps=500,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [52]:
train_dataset = Dataset.from_dict({"input_data": X_train, "sentiment": y_train})
eval_dataset = Dataset.from_dict({"input_data": X_test, "sentiment": y_test})

In [53]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)
trainer.train()

c:\Users\Assistant\anaconda3\envs\dev\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/5508 [00:00<?, ?it/s]

IndexError: Invalid key: 57300 is out of bounds for size 0

In [ ]:
results = trainer.evaluate()


{'eval_runtime': 0.0172, 'eval_samples_per_second': 0.0, 'eval_steps_per_second': 0.0, 'epoch': 0}


In [ ]:
predictions = trainer.predict(your_inference_dataset)